In [ ]:
!pip -q install pandas tqdm trafilatura
!pip install feedparser

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 837.9/837.9 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 15.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 5.0 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=a367923a59f35660d6d8126bfb6baae8736596cf181d24db4530985ff94256a8
  Stored in directory: /root/.cache/pip/wheels/03/f5/1a/23761066dac1d0e8e683e5fdb27e12de53209d05a4a37e6246
Successfully built sgmllib3k


In [ ]:
import feedparser
import pandas as pd
import re

subreddits = [
    "VisitingIceland", #r/Iceland, r/VistingIceland, r/LivingIceland
    "glutenfree",
    "Celiac",
    "camping",
    "backpacking",
    "vanlife",
    "EatCheapAndHealthy",
    "MealPrepSunday"
]

PAIN_RE = re.compile(r"(can't|cannot|struggle|hard|difficult|expensive|allerg|celiac|gluten|dairy|nut|no kitchen|no microwave|travel|camping|hostel|hotel)", re.I)

rows=[]
for sr in subreddits:
    feed_url = f"https://www.reddit.com/r/{sr}/.rss"
    d = feedparser.parse(feed_url)
    for e in d.entries:
        text = (e.get("title","") + " " + e.get("summary","")).strip()
        if PAIN_RE.search(text):
            rows.append({
                "source": f"reddit:rss:{sr}",
                "url": e.get("link",""),
                "title": e.get("title",""),
                "pain_sentence": e.get("title","")  # RSS mostly gives post title as the clearest pain point
            })

rdf = pd.DataFrame(rows).drop_duplicates()
rdf.to_csv("pain_points_reddit_rss.csv", index=False)
rdf.head(20), len(rdf)


(                        source  \
 0   reddit:rss:VisitingIceland   
 1   reddit:rss:VisitingIceland   
 2   reddit:rss:VisitingIceland   
 3   reddit:rss:VisitingIceland   
 4   reddit:rss:VisitingIceland   
 5   reddit:rss:VisitingIceland   
 6   reddit:rss:VisitingIceland   
 7   reddit:rss:VisitingIceland   
 8   reddit:rss:VisitingIceland   
 9   reddit:rss:VisitingIceland   
 10  reddit:rss:VisitingIceland   
 11       reddit:rss:glutenfree   
 12       reddit:rss:glutenfree   
 13       reddit:rss:glutenfree   
 14       reddit:rss:glutenfree   
 15       reddit:rss:glutenfree   
 16       reddit:rss:glutenfree   
 17       reddit:rss:glutenfree   
 18       reddit:rss:glutenfree   
 19       reddit:rss:glutenfree   
 
                                                   url  \
 0   https://www.reddit.com/r/VisitingIceland/comme...   
 1   https://www.reddit.com/r/VisitingIceland/comme...   
 2   https://www.reddit.com/r/VisitingIceland/comme...   
 3   https://www.reddit.com/r/V

In [ ]:
import re

BAD_TITLE_RE = re.compile(
    r"(megathread|faq|weekly|monthly|thread|announcement|"
    r"i finally got|finally got|dinner is served|new item|"
    r"amazing|mid\?!|so similar|look what i found|"
    r"travel partners|eclipse|live music|traffic rule|core memories)",
    re.I
)

GOOD_SIGNAL_RE = re.compile(
    r"(help|tips|recommend|any advice|where can i|what do i|"
    r"budget|expensive|cost|cheap|afford|"
    r"camp|camper|van|road trip|hostel|hotel|"
    r"no kitchen|no microwave|no stove|no fridge|"
    r"celiac|gluten|dairy|nut|allerg|intoleran|"
    r"easy meal|quick meal|one pot|meal kit)",
    re.I
)

df2 = rdf.copy()
df2 = df2[~df2["title"].fillna("").str.contains(BAD_TITLE_RE)]
df2 = df2[df2["title"].fillna("").str.contains(GOOD_SIGNAL_RE)]

df2 = df2.drop_duplicates(subset=["url"]).reset_index(drop=True)
df2.head(20), len(df2)

/tmp/ipython-input-3245515995.py:22: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df2 = df2[~df2["title"].fillna("").str.contains(BAD_TITLE_RE)]
/tmp/ipython-input-3245515995.py:23: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df2 = df2[df2["title"].fillna("").str.contains(GOOD_SIGNAL_RE)]


(                        source  \
 0   reddit:rss:VisitingIceland   
 1   reddit:rss:VisitingIceland   
 2   reddit:rss:VisitingIceland   
 3        reddit:rss:glutenfree   
 4        reddit:rss:glutenfree   
 5        reddit:rss:glutenfree   
 6        reddit:rss:glutenfree   
 7        reddit:rss:glutenfree   
 8        reddit:rss:glutenfree   
 9        reddit:rss:glutenfree   
 10       reddit:rss:glutenfree   
 11           reddit:rss:Celiac   
 12           reddit:rss:Celiac   
 13           reddit:rss:Celiac   
 14           reddit:rss:Celiac   
 15           reddit:rss:Celiac   
 16           reddit:rss:Celiac   
 17          reddit:rss:camping   
 18          reddit:rss:camping   
 19          reddit:rss:camping   
 
                                                   url  \
 0   https://www.reddit.com/r/VisitingIceland/comme...   
 1   https://www.reddit.com/r/VisitingIceland/comme...   
 2   https://www.reddit.com/r/VisitingIceland/comme...   
 3   https://www.reddit.com/r/g

In [ ]:
FOOD_MEAL_RE = re.compile(
    r"(meal|dinner|lunch|breakfast|eat|cook|food|rice|one[- ]?pot|"
    r"instant|quick|easy|kit|camp|camper|van|travel|hotel|hostel|"
    r"no kitchen|no microwave|no stove|road trip)",
    re.I
)

df3 = df2[df2["title"].str.contains(FOOD_MEAL_RE, na=False)]
df3 = df3.reset_index(drop=True)

df3[["title","url"]].head(15), len(df3)

/tmp/ipython-input-1045557783.py:8: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df3 = df2[df2["title"].str.contains(FOOD_MEAL_RE, na=False)]


(                                                title  \
 0                                   Camper van rental   
 1   Potential gluten contamination in Vans gluten ...   
 2                     Would u guys eat this? (Celiac)   
 3              gluten free chocolate chip cookie milk   
 4   Would you eat vegetables that were put on a co...   
 5                  Looking for help. Mountain camping   
 6            New to this subreddit but not to camping   
 7   upgraded my overlanding rig and now i actually...   
 8   Camping is the best reset and I don’t think an...   
 9                               Feb camping in S Ohio   
 10                           Camping am Ossiacher See   
 11                         charcoal campfire question   
 12                         Generator solution camping   
 13       2 or 3 seat heated camping chair suggestions   
 14                               Winter Camping Tips?   
 
                                                   url  
 0   https://

In [ ]:
def normalize_pain(title):
    t = title.strip()

    if re.search(r"instant rice", t, re.I):
        return "I need quick, instant meals that are safe for dietary restrictions."

    if re.search(r"camper|van", t, re.I):
        return "I’m traveling in a camper van and need easy meals with minimal cooking equipment."

    if re.search(r"no kitchen|no microwave", t, re.I):
        return "I don’t have access to a kitchen and still need proper meals."

    if re.search(r"gluten", t, re.I):
        return "Finding safe, gluten-free meals that are quick and reliable is difficult."

    if re.search(r"itinerary|travel", t, re.I):
        return "While traveling, I’m unsure how to handle meals without overspending or cooking access."

    return t

df3["normalized_pain"] = df3["title"].apply(normalize_pain)
df3[["title","normalized_pain"]].head(10)

,title,normalized_pain
0,Camper van rental,I’m traveling in a camper van and need easy me...
1,Potential gluten contamination in Vans gluten ...,I’m traveling in a camper van and need easy me...
2,Would u guys eat this? (Celiac),Would u guys eat this? (Celiac)
3,gluten free chocolate chip cookie milk,"Finding safe, gluten-free meals that are quick..."
4,Would you eat vegetables that were put on a co...,"Finding safe, gluten-free meals that are quick..."
5,Looking for help. Mountain camping,Looking for help. Mountain camping
6,New to this subreddit but not to camping,New to this subreddit but not to camping
7,upgraded my overlanding rig and now i actually...,upgraded my overlanding rig and now i actually...
8,Camping is the best reset and I don’t think an...,Camping is the best reset and I don’t think an...
9,Feb camping in S Ohio,Feb camping in S Ohio


In [ ]:
def kays_fit(p):
    p = p.lower()
    feats = []

    if any(w in p for w in ["no kitchen","camper","van","hotel","hostel"]):
        feats.append("one-pot, minimal equipment")

    if any(w in p for w in ["quick","instant","easy"]):
        feats.append("fast prep (15–20 min)")

    if "gluten" in p:
        feats.append("gluten-free & allergen-safe")

    if any(w in p for w in ["travel","road","trip"]):
        feats.append("portable & shelf-stable")

    return ", ".join(feats)

df3["kays_solution"] = df3["normalized_pain"].apply(kays_fit)

# 1 = has solution, 0 = no solution
df3["kays_priority"] = df3["kays_solution"].apply(lambda x: 1 if x.strip() else 0)

df3[["normalized_pain","kays_solution"]].head(20)

,normalized_pain,kays_solution
0,I’m traveling in a camper van and need easy me...,"one-pot, minimal equipment, fast prep (15–20 m..."
1,I’m traveling in a camper van and need easy me...,"one-pot, minimal equipment, fast prep (15–20 m..."
2,Would u guys eat this? (Celiac),
3,"Finding safe, gluten-free meals that are quick...","fast prep (15–20 min), gluten-free & allergen-..."
4,"Finding safe, gluten-free meals that are quick...","fast prep (15–20 min), gluten-free & allergen-..."
5,Looking for help. Mountain camping,
6,New to this subreddit but not to camping,
7,upgraded my overlanding rig and now i actually...,
8,Camping is the best reset and I don’t think an...,
9,Feb camping in S Ohio,


In [ ]:
df3 = df3.sort_values(
    by=["kays_priority", "normalized_pain"],
    ascending=[False, True]
).reset_index(drop=True)
df3

,source,url,title,pain_sentence,normalized_pain,kays_solution,kays_priority
0,reddit:rss:MealPrepSunday,https://www.reddit.com/r/MealPrepSunday/commen...,"Easy and cheap, low carb pesto chicken bowls!","Easy and cheap, low carb pesto chicken bowls!","Easy and cheap, low carb pesto chicken bowls!",fast prep (15–20 min),1
1,reddit:rss:glutenfree,https://www.reddit.com/r/glutenfree/comments/1...,gluten free chocolate chip cookie milk,gluten free chocolate chip cookie milk,"Finding safe, gluten-free meals that are quick...","fast prep (15–20 min), gluten-free & allergen-...",1
2,reddit:rss:Celiac,https://www.reddit.com/r/Celiac/comments/1qojh...,Would you eat vegetables that were put on a co...,Would you eat vegetables that were put on a co...,"Finding safe, gluten-free meals that are quick...","fast prep (15–20 min), gluten-free & allergen-...",1
3,reddit:rss:VisitingIceland,https://www.reddit.com/r/VisitingIceland/comme...,Camper van rental,Camper van rental,I’m traveling in a camper van and need easy me...,"one-pot, minimal equipment, fast prep (15–20 m...",1
4,reddit:rss:glutenfree,https://www.reddit.com/r/glutenfree/comments/1...,Potential gluten contamination in Vans gluten ...,Potential gluten contamination in Vans gluten ...,I’m traveling in a camper van and need easy me...,"one-pot, minimal equipment, fast prep (15–20 m...",1
5,reddit:rss:vanlife,https://www.reddit.com/r/VanLife/comments/1qop...,Simple Hack: House sit while campervanning,Simple Hack: House sit while campervanning,I’m traveling in a camper van and need easy me...,"one-pot, minimal equipment, fast prep (15–20 m...",1
6,reddit:rss:vanlife,https://www.reddit.com/r/VanLife/comments/1qoc...,Camper Rental in Montenegro – Everything You N...,Camper Rental in Montenegro – Everything You N...,I’m traveling in a camper van and need easy me...,"one-pot, minimal equipment, fast prep (15–20 m...",1
7,reddit:rss:vanlife,https://www.reddit.com/r/VanLife/comments/1qnb...,Building a Bigger DIY Camper for Our Growing F...,Building a Bigger DIY Camper for Our Growing F...,I’m traveling in a camper van and need easy me...,"one-pot, minimal equipment, fast prep (15–20 m...",1
8,reddit:rss:vanlife,https://www.reddit.com/r/VanLife/comments/1qmz...,Vanlifers what’s the scariest/eeriest encounte...,Vanlifers what’s the scariest/eeriest encounte...,I’m traveling in a camper van and need easy me...,"one-pot, minimal equipment, fast prep (15–20 m...",1
9,reddit:rss:backpacking,https://www.reddit.com/r/backpacking/comments/...,Is the Red Sea in Egypt good for snorkeling an...,Is the Red Sea in Egypt good for snorkeling an...,"While traveling, I’m unsure how to handle meal...",portable & shelf-stable,1


In [ ]:
df_kays_only = df3[df3["kays_priority"] == 1]
df_kays_only.head(20)

,source,url,title,pain_sentence,normalized_pain,kays_solution,kays_priority
0,reddit:rss:MealPrepSunday,https://www.reddit.com/r/MealPrepSunday/commen...,"Easy and cheap, low carb pesto chicken bowls!","Easy and cheap, low carb pesto chicken bowls!","Easy and cheap, low carb pesto chicken bowls!",fast prep (15–20 min),1
1,reddit:rss:glutenfree,https://www.reddit.com/r/glutenfree/comments/1...,gluten free chocolate chip cookie milk,gluten free chocolate chip cookie milk,"Finding safe, gluten-free meals that are quick...","fast prep (15–20 min), gluten-free & allergen-...",1
2,reddit:rss:Celiac,https://www.reddit.com/r/Celiac/comments/1qojh...,Would you eat vegetables that were put on a co...,Would you eat vegetables that were put on a co...,"Finding safe, gluten-free meals that are quick...","fast prep (15–20 min), gluten-free & allergen-...",1
3,reddit:rss:VisitingIceland,https://www.reddit.com/r/VisitingIceland/comme...,Camper van rental,Camper van rental,I’m traveling in a camper van and need easy me...,"one-pot, minimal equipment, fast prep (15–20 m...",1
4,reddit:rss:glutenfree,https://www.reddit.com/r/glutenfree/comments/1...,Potential gluten contamination in Vans gluten ...,Potential gluten contamination in Vans gluten ...,I’m traveling in a camper van and need easy me...,"one-pot, minimal equipment, fast prep (15–20 m...",1
5,reddit:rss:vanlife,https://www.reddit.com/r/VanLife/comments/1qop...,Simple Hack: House sit while campervanning,Simple Hack: House sit while campervanning,I’m traveling in a camper van and need easy me...,"one-pot, minimal equipment, fast prep (15–20 m...",1
6,reddit:rss:vanlife,https://www.reddit.com/r/VanLife/comments/1qoc...,Camper Rental in Montenegro – Everything You N...,Camper Rental in Montenegro – Everything You N...,I’m traveling in a camper van and need easy me...,"one-pot, minimal equipment, fast prep (15–20 m...",1
7,reddit:rss:vanlife,https://www.reddit.com/r/VanLife/comments/1qnb...,Building a Bigger DIY Camper for Our Growing F...,Building a Bigger DIY Camper for Our Growing F...,I’m traveling in a camper van and need easy me...,"one-pot, minimal equipment, fast prep (15–20 m...",1
8,reddit:rss:vanlife,https://www.reddit.com/r/VanLife/comments/1qmz...,Vanlifers what’s the scariest/eeriest encounte...,Vanlifers what’s the scariest/eeriest encounte...,I’m traveling in a camper van and need easy me...,"one-pot, minimal equipment, fast prep (15–20 m...",1
9,reddit:rss:backpacking,https://www.reddit.com/r/backpacking/comments/...,Is the Red Sea in Egypt good for snorkeling an...,Is the Red Sea in Egypt good for snorkeling an...,"While traveling, I’m unsure how to handle meal...",portable & shelf-stable,1
